In [3]:
import comet_ml
from config import SAVED_MODEL

comet_ml.init(api_key="wfYwSiDlqTHVyTWHjAuT6qI0P")

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline, BertForMaskedLM, BertTokenizer
from transformers import BertModel
from datasets import load_dataset

from torch.utils.data import DataLoader

from transformers import Trainer, TrainingArguments
from datetime import datetime


COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()
COMET INFO: Valid Comet API Key saved in /home/henryp/.comet.config (set COMET_CONFIG to change where it is saved).


Model

In [4]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

Data


In [5]:
dataset = load_dataset("imdb")  # or your own dataset
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")  # required by HF
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

train_loader = DataLoader(tokenized_dataset['train'], batch_size=8, shuffle=True)
val_loader = DataLoader(tokenized_dataset['test'], batch_size=8)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    report_to="comet_ml",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
)

timestamp = datetime.now().strftime('%y%m%d%H%M')
model_path = f"{SAVED_MODEL}/{timestamp}/{model_name}"
print(f"Will train model and save it to {model_path}")
trainer.train()



/home/henryp/venv/llms/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch, keras, tensorflow.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Will train model and save it to ./saved_model/2505071143/bert-base-uncased


COMET INFO: Experiment is live on comet.com https://www.comet.com/phillhenry/general/f3f125fa11e34b3fbb41948b4a398fd7



Epoch,Training Loss,Validation Loss
1,2.018000,1.896111


TrainOutput(global_step=3125, training_loss=2.09102556640625, metrics={'train_runtime': 2200.8368, 'train_samples_per_second': 11.359, 'train_steps_per_second': 1.42, 'total_flos': 6580120320000000.0, 'train_loss': 2.09102556640625, 'epoch': 1.0})

In [7]:
comet_ml.experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : olympic_arch_4399
COMET INFO:     url                   : https://www.comet.com/phillhenry/general/f3f125fa11e34b3fbb41948b4a398fd7
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     eval/loss                      : 1.8961111307144165
COMET INFO:     eval/runtime                   : 529.9033
COMET INFO:     eval/samples_per_second        : 47.178
COMET INFO:     eval/steps_per_second          : 5.897
COMET INFO:     train/epoch [8]                : (0.16, 1.0)
COMET INFO:     train/grad_norm [6]            : (5.760892391204834, 7.740053653717041)
COMET INFO:     train/learning_rate [6]        : (2.0000000000000003e-06, 4.2e-05)
COMET INFO:    

In [8]:
#model_path = f"{MyLlamaModel.base_output_dir}/{model_name}"

model.save_pretrained(model_path)
tokenizer.save_pretrained(f"{model_path}_tokenizer")
tokenizer.save_vocabulary(f"{model_path}_vocab")
#model.save_pretrained_merged(model_path, tokenizer=tokenizer) # merged_4bit_forced
print("finished")

finished


In [10]:
unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)
print(unmasker("Hello I'm a [MASK] model."))

Device set to use cuda:0


[{'score': 0.19769975543022156, 'token': 2535, 'token_str': 'role', 'sequence': "hello i'm a role model."}, {'score': 0.0841338112950325, 'token': 4827, 'token_str': 'fashion', 'sequence': "hello i'm a fashion model."}, {'score': 0.04360273852944374, 'token': 2307, 'token_str': 'great', 'sequence': "hello i'm a great model."}, {'score': 0.04252929985523224, 'token': 3287, 'token_str': 'male', 'sequence': "hello i'm a male model."}, {'score': 0.03406408801674843, 'token': 3565, 'token_str': 'super', 'sequence': "hello i'm a super model."}]


In [11]:
print(unmasker("The Empire Strikes [MASK]"))

[{'score': 0.9441056251525879, 'token': 999, 'token_str': '!', 'sequence': 'the empire strikes!'}, {'score': 0.05120796710252762, 'token': 1012, 'token_str': '.', 'sequence': 'the empire strikes.'}, {'score': 0.003980387933552265, 'token': 1025, 'token_str': ';', 'sequence': 'the empire strikes ;'}, {'score': 0.000549117277842015, 'token': 1029, 'token_str': '?', 'sequence': 'the empire strikes?'}, {'score': 4.1509574657538906e-05, 'token': 1064, 'token_str': '|', 'sequence': 'the empire strikes |'}]
